# Non Lineair Dynamics Project

In this project we explore the stability of biological communities. We will be studying the effect when we combine mutalistic and antognistic interaction between species together in one community.

More concreet we will try to reproduce the results of this paper: *A. Moughi and M. Kondoh:* Diversity of Interaction Types and Ecological Community Stability.*Science* **337**,349 (2012)

## Methodology

First some basic questions out the way:
* **Q:What exactly do we want to study?**

*A: We want to study the stability of a complex community. For this we will actually check for a given fixed set of parameters, that define the community dynamics, if it is stable or not. Doing this then many times for the same configuration we can eventually put a probability on how stable the community is under these parameters.*
* **Q:How do we even represent a community and its dynamics?**

*A: We will be working with a vector $\vec{X}$ who's components are the abundance of species, and thus represent the state of our community at a given time. The dynamics of this vector $ \vec{X} $ are goverend by a coupled set of differential equations looking of the form: $$\frac{d\vec{X}}{dt}=\vec{F}(\vec{X},\vec{\alpha})$$ Here $\vec{F}$ is a vector of function of which each depends on $\vec{X}$ and $\vec{\alpha}$. $\vec{\alpha}$ is a vector of tunable parameter values that determine the dynamics of the system.*
* **Q:How do we know if the community is stable?**

*A: We will do linear stability analysis around a fixed point of the community to determine the stability. For this we need to determine the Jacobian Matrix of the dynamic equation and evaluate it at that point. We know the point is stable if the real part of the eigenvalues are all smaller or equal to zero. This is easily understood if one looks at the general solution of a linearised system:$$\sum_{k=1}^{N} c_ke^{\lambda_k t}\vec{v_k}$$ Where k sums over all the eigenvectors $\vec{v_k}$ and eigenvalues $\lambda_k$, and $N$ is the dimension of the system, and $c_k$ some constante to be determined by the intial condition. If one of the eigenvalues $\lambda_k$ has a real positif part. The solution will then always grow exponentially and move away from the fixed point allong the $\vec{v_k}$ direction.*
* **Q:How do we even know that there is a fixed point let alone how to find it for a big system?**

*A: To solve both problems we will **impose** a fixed point in our community. This will be done by tuning the growth rate $r_i$ (see later), such that the fixed point is created. From here on out we can carry out our analysis.*
* **Q:And other fixed points that might be stable?**

*A: Indeed it is possible that the fixed point that we imposed is not stable. And that when we move away from our point and end up stumbeling on another fixed point that is stable. We then still classified our system as unstable, since we only evaluate our point. This can indeed happen and there is no way for us knowing if this happens. But more then likely is that the other fixed point is stable but with a whole lot of species dead (some $\vec{X}$ components =0), we then also see the system as unstable since we want to look at stability that can support **all** our species.*

## Choice of our Dynamical Model

*Work in progress*
Explain here how the dynamical model looks like and why it is motivated this way. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy